In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize']=20,10
from keras.models import Sequential
from keras.layers import LSTM,Dropout,Dense
from sklearn.preprocessing import MinMaxScaler
import yfinance as yf
import talib as ta
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

from keras.models import Sequential
from keras.layers import Dense, Activation, Embedding, Flatten, LeakyReLU, BatchNormalization, Dropout
from keras.activations import relu, sigmoid
from keras.layers import LeakyReLU

from datetime import datetime

from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch

ModuleNotFoundError: No module named 'talib'

### Importing the Data of the Stock/Index

In [158]:
df = yf.download('RELIANCE.NS', '2008-01-01', auto_adjust=True)

df = df.reset_index()

[*********************100%***********************]  1 of 1 completed


### Adding the technical indicators and any relevant parameters

In [159]:
def get_wr(high, low, close, lookback):
    highh = high.rolling(lookback).max() 
    lowl = low.rolling(lookback).min()
    wr = -100 * ((highh - close) / (highh - lowl))
    return wr

In [160]:
def ROC(data,n):
    N = data['Close'].diff(n)
    D = data['Close'].shift(n)
    ROC = pd.Series(N/D,name='Rate of Change')
    data = data.join(ROC)
    return data 

In [161]:
df['SMA_5'] = df['Close'].transform(lambda x: x.rolling(window = 5).mean())
df['SMA_15'] = df['Close'].transform(lambda x: x.rolling(window = 15).mean())
df['SMA_ratio'] = df['SMA_15'] / df['SMA_5']

df['SMA5_Volume'] = df['Volume'].transform(lambda x: x.rolling(window = 5).mean())
df['SMA15_Volume'] = df['Volume'].transform(lambda x: x.rolling(window = 15).mean())
df['SMA_Volume_Ratio'] = df['SMA5_Volume']/df['SMA15_Volume']

df['SMA_20'] = ta.SMA(df['Close'],20)
df['EMA_20'] = ta.EMA(df['Close'], timeperiod = 20)
df['ADX'] = ta.ADX(df['High'],df['Low'], df['Close'], timeperiod=20)

df['up_band'], df['mid_band'], df['low_band'] = ta.BBANDS(df['Close'], timeperiod =20)

df['RSI'] = ta.RSI(df['Close'],14)

df['wr_14'] = get_wr(df['High'],df['Low'], df['Close'], 14)


NIFTY_ROC = ROC(df,5)
df['ROC'] = NIFTY_ROC['Rate of Change']

# aapl_df.ta.log_return(cumulative=True, append=True)
# aapl_df.ta.percent_return(cumulative=True, append=True)
# aapl_df.ta.vwap(cumulative=True, append=True)
# aapl_df.ta.atr(cumulative=True, append=True)
# aapl_df.ta.smi(cumulative=True, append=True)
# aapl_df.ta.adx(cumulative=True, append=True)
# aapl_df.ta.aroon(cumulative=True, append=True)
# aapl_df.ta.bbands(cumulative=True, append=True)

# aapl_df.ta.cmf(cumulative=True, append=True)
# # aapl_df.ta.psar(cumulative=True, append=True)
# aapl_df.ta.mfi(cumulative=True, append=True)
# aapl_df.ta.macd(cumulative=True, append=True)
# # aapl_df.ta.eom(cumulative=True, append=True)
# aapl_df.ta.rsi(cumulative=True, append=True)

# aapl_df.ta.ema(close=aapl_df.ta.ohlc4(), length=10, suffix="OHLC4",append = True)
# aapl_df.ta.sma(close=aapl_df.ta.ohlc4(), length=10, suffix="OHLC4",append = True)
# aapl_df.ta.ema(close=aapl_df.ta.ohlc4(), length=20, suffix="OHLC4",append = True)
# aapl_df.ta.sma(close=aapl_df.ta.ohlc4(), length=20, suffix="OHLC4",append = True)
# aapl_df.ta.ema(close=aapl_df.ta.ohlc4(), length=30, suffix="OHLC4",append = True)
# aapl_df.ta.sma(close=aapl_df.ta.ohlc4(), length=30, suffix="OHLC4",append = True)
# aapl_df.ta.ema(close=aapl_df.ta.ohlc4(), length=50, suffix="OHLC4",append = True)
# aapl_df.ta.sma(close=aapl_df.ta.ohlc4(), length=50, suffix="OHLC4",append = True)
# aapl_df.ta.ema(close=aapl_df.ta.ohlc4(), length=100, suffix="OHLC4",append = True)
# aapl_df.ta.sma(close=aapl_df.ta.ohlc4(), length=100, suffix="OHLC4",append = True)
# aapl_df.ta.ema(close=aapl_df.ta.ohlc4(), length=200, suffix="OHLC4",append = True)
# aapl_df.ta.sma(close=aapl_df.ta.ohlc4(), length=200, suffix="OHLC4",append = True)

# aapl_df.ta.vwma(cumulative=True,length = 20, append=True)
# aapl_df.ta.hma(cumulative=True,length = 9, append=True)

# aapl_df['SAR'] = talib.SAR(aapl_df.High, aapl_df.Low, acceleration=0.02, maximum=0.2)

# aapl_df['avg_price'] = (aapl_df['Open'] + aapl_df['Close'])/2




# df['Next_day_Close'] = df['Close'].shift(-1)
df['RSI'] = ta.RSI(np.array(df['Close']), timeperiod =20)
df['Date_of_week']= df['Date'].dt.dayofweek

df['Next_day_Close_Side'] = np.where(df['Close'].shift(-1) > df['Close'],1,0)

In [164]:
df.columns

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'SMA_5', 'SMA_15',
       'SMA_ratio', 'SMA5_Volume', 'SMA15_Volume', 'SMA_Volume_Ratio',
       'SMA_20', 'EMA_20', 'ADX', 'up_band', 'mid_band', 'low_band', 'RSI',
       'wr_14', 'ROC', 'Date_of_week', 'Next_day_Close_Side'],
      dtype='object')

### Data preparation and pre processing

In [163]:
df = df.dropna()
df.reset_index(inplace = True, drop = True)

### Extracting the Dependent and Independent Variables

In [177]:
X = df.iloc[:, 1:22]
y = df.iloc[:, 22]

In [178]:
X

,Open,High,Low,Close,Volume,SMA_5,SMA_15,SMA_ratio,SMA5_Volume,SMA15_Volume,...,SMA_20,EMA_20,ADX,up_band,mid_band,low_band,RSI,wr_14,ROC,Date_of_week
0,555.650381,571.239732,536.731539,568.222107,7484576,557.688245,552.279451,0.990301,7478165.2,1.060780e+07,...,555.200610,566.246533,22.438672,594.858263,555.200610,515.542958,47.529964,-17.376175,-0.000881,0
1,571.217564,575.916649,563.912781,573.834351,5903324,558.672583,551.678878,0.987482,7359696.0,1.053519e+07,...,555.197818,566.969182,22.032732,594.850214,555.197818,515.545422,48.472255,-10.796471,0.008651,1
2,580.147982,587.497278,572.364421,576.462219,10762911,563.324878,552.212622,0.980274,7729457.6,1.066002e+07,...,556.495630,567.873281,21.413778,597.135604,556.495630,515.855656,48.924351,-12.295298,0.042049,2
3,574.580498,579.034567,560.483188,564.169006,11394437,564.970679,553.813135,0.980251,8710168.8,1.057611e+07,...,557.100839,567.520493,21.016492,597.819289,557.100839,516.382390,46.898145,-25.992436,0.014802,3
4,568.344789,568.344789,536.720614,548.624207,13474162,566.262378,554.363224,0.978987,9803882.0,1.062365e+07,...,556.217816,565.720847,20.992245,596.867236,556.217816,515.568396,44.447920,-45.813424,0.011912,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3430,2360.199951,2406.399902,2356.199951,2386.600098,5604308,2362.340039,2456.773372,1.039974,8603371.0,6.740675e+06,...,2451.782544,2426.827842,16.415562,2583.833859,2451.782544,2319.731229,45.825402,-68.890552,-0.036826,0
3431,2408.000000,2420.600098,2347.100098,2378.699951,6092033,2362.500049,2452.820036,1.038231,7803275.4,6.862319e+06,...,2450.525037,2422.244233,16.201236,2584.837857,2450.525037,2316.212217,45.184504,-71.902426,0.000336,1
3432,2407.000000,2407.000000,2375.100098,2383.550049,4623163,2364.560059,2448.020036,1.035296,6465616.8,6.671951e+06,...,2446.797534,2418.559073,15.997625,2584.165689,2446.797534,2309.429379,45.675491,-70.053361,0.004340,2
3433,2392.000000,2401.449951,2347.199951,2352.750000,4417864,2367.490039,2436.796696,1.029274,5501304.4,6.511116e+06,...,2440.955029,2412.291542,16.056196,2583.778231,2440.955029,2298.131828,43.095177,-81.795657,0.006266,3


### Creating the Training and Test Datasets

In [179]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

### Scaling the data between 0 and 1 

In [180]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

### Function to iterate over the parameters and return the best results

In [181]:
def create_model(layers, activation):
    model = Sequential()
    for i, nodes in enumerate(layers):
        if i==0:
            model.add(Dense(nodes,input_dim=X_train.shape[1]))
            model.add(Activation(activation))
        else:
            model.add(Dense(nodes))
            model.add(Activation(activation))
    model.add(Dense(1)) # Note: no activation beyond this point
    
    model.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy'])
    return model
    
model = KerasClassifier(build_fn=create_model, verbose=0)

/var/folders/gp/x0f5sgq13db073j79l3_2tb40000gn/T/ipykernel_22605/3582075148.py:15: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=create_model, verbose=0)


### Inititalise the parameters you want to pass

In [182]:
layers = [[20], [40, 20], [45, 30, 15]]
activations = ['sigmoid', 'relu']
param_grid = dict(layers=layers, activation=activations, batch_size = [128, 256], epochs=[30])
grid = GridSearchCV(estimator=model, param_grid=param_grid)

### Perform the grid search operation to get the results

In [183]:
grid_result = grid.fit(X_train, y_train)

### Visualise the best outcome

In [184]:
[grid_result.best_score_,grid_result.best_params_]

[0.5083649516105652,
 {'activation': 'relu',
  'batch_size': 128,
  'epochs': 30,
  'layers': [45, 30, 15]}]

In [99]:
pred_y = grid.predict(X_test)

In [100]:
y_pred = (pred_y > 0.5)

In [102]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

from sklearn.metrics import accuracy_score
score=accuracy_score(y_test,y_pred)

score

0.507959479015919